In [81]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

# Database connections 

In [82]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [83]:

# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# SQL Query

In [84]:
query = """
SELECT 
    s.id as solicitud_id,
    s.fecha_solicitud,
    s.hora_solicitud::time as hora,
    s.cliente_id,
    sede.sede_id,
    COALESCE(s.mensajero3_id, COALESCE(s.mensajero2_id, s.mensajero_id)) AS mensajero_final_id,
    COUNT(*) as cantidad_servicios
FROM mensajeria_servicio s
JOIN mensajeria_estadosservicio es 
    ON s.id = es.servicio_id
JOIN mensajeria_origenservicio os 
    ON s.origen_id = os.id
JOIN sede 
    ON sede.cliente_id = os.cliente_id 
    AND sede.ciudad_id = os.ciudad_id
GROUP BY 
    solicitud_id,
    fecha_solicitud,
    hora,
    s.cliente_id,
    sede.sede_id,
    mensajero_final_id
ORDER BY 
    s.fecha_solicitud, hora;
"""

# Extract

In [85]:
# Leer datos y dimensiones
df = pd.read_sql(query, fuente_conn)
dim_fecha = pd.read_sql_table('dim_fecha', bodega_conn)
dim_cliente = pd.read_sql_table('dim_cliente', bodega_conn)
dim_mensajero = pd.read_sql_table('dim_mensajero', bodega_conn)
dim_sede = pd.read_sql_table('dim_sede', bodega_conn)
dim_hora = pd.read_sql_table('dim_hora', bodega_conn)

In [86]:
# Limpiar formato de hora
def limpiar_hora(hora_str):
    try:
        if '.' in str(hora_str):
            return str(hora_str).split('.')[0]
        return str(hora_str)
    except:
        return None

df['hora'] = df['hora'].apply(limpiar_hora)


In [87]:
# Extraer componentes de hora
def extraer_hora(hora_str):
    try:
        if pd.isna(hora_str):
            return None
        return int(hora_str.split(':')[0])
    except:
        return None

def extraer_minuto(hora_str):
    try:
        if pd.isna(hora_str):
            return None
        return int(hora_str.split(':')[1])
    except:
        return None

def extraer_segundo(hora_str):
    try:
        if pd.isna(hora_str):
            return None
        return int(hora_str.split(':')[2])
    except:
        return None

df['hora'] = df['hora'].apply(extraer_hora)
df['minuto'] = df['hora'].apply(extraer_minuto)
df['segundo'] = df['hora'].apply(extraer_segundo)


# Transformations

In [88]:
# Convertir fecha_estado a datetime
df['fecha_solicitud'] = pd.to_datetime(df['fecha_solicitud']).dt.date
dim_fecha['fecha'] = pd.to_datetime(dim_fecha['fecha']).dt.date

In [89]:
# Realizar los merges con las dimensiones
hecho_hora = df.merge(
    dim_fecha[['key_dim_fecha', 'fecha']], 
    left_on='fecha_solicitud', 
    right_on='fecha',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_cliente[['key_dim_cliente', 'cliente_id']], 
    on='cliente_id',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_mensajero[['key_dim_mensajero', 'mensajero_id']], 
    left_on='mensajero_final_id',
    right_on='mensajero_id',
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_sede[['key_dim_sede', 'id_sede']], 
    left_on='sede_id',                 
    right_on='id_sede',                
    how='left'
)

hecho_hora = hecho_hora.merge(
    dim_hora[['key_dim_hora', 'hora']],
    on='hora',
    how='left'
)

In [90]:
# Calcular tiempo total por hora
def calcular_tiempo_total(grupo):
    return len(grupo)  # Cantidad de servicios por hora

hecho_hora_agrupado = hecho_hora.groupby([
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_sede',
    'key_dim_mensajero',
    'key_dim_hora'
]).apply(calcular_tiempo_total).reset_index(name='hora_total')

C:\Users\HP\AppData\Local\Temp\ipykernel_18088\4227800117.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ]).apply(calcular_tiempo_total).reset_index(name='hora_total')


In [91]:
# Agregar fecha de carga
hecho_hora_agrupado['saved'] = date.today()

# Verifications

In [92]:
# Verificaciones
print("\nInformación del DataFrame:")
print(hecho_hora_agrupado.info())

print("\nDistribución de servicios por hora:")
print(hecho_hora_agrupado.groupby('key_dim_hora')['hora_total'].mean())

print("\nVerificar valores nulos:")
print(hecho_hora_agrupado.isnull().sum())



Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115637 entries, 0 to 115636
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key_dim_fecha      115637 non-null  int64  
 1   key_dim_cliente    115637 non-null  int64  
 2   key_dim_sede       115637 non-null  int64  
 3   key_dim_mensajero  115637 non-null  float64
 4   key_dim_hora       115637 non-null  int64  
 5   hora_total         115637 non-null  int64  
 6   saved              115637 non-null  object 
dtypes: float64(1), int64(5), object(1)
memory usage: 6.2+ MB
None

Distribución de servicios por hora:
key_dim_hora
0     4.615108
1     4.812903
2     4.850163
3     4.412556
4     4.575107
5     4.239130
6     6.820022
7     5.772283
8     7.255247
9     7.732807
10    7.418313
11    8.056496
12    6.629150
13    6.607655
14    7.546015
15    7.656119
16    7.179348
17    6.212675
18    4.852231
19    5.476056
20  

# Load

In [93]:
#Guardar en la bodega
hecho_hora_agrupado.to_sql(
    'hecho_entrega_servicio_hora', 
    bodega_conn, 
    if_exists='replace', 
    index_label='key_hecho_entrega_hora'
)

637

# SQL Querys to response requirements

In [94]:
#Pregunta 3: ¿A qué hora los mensajeros están más ocupados?
'''
SELECT 
    dh.hora AS hora_del_dia,
    SUM(h.hora_total) AS total_servicios
FROM hecho_entrega_servicio_hora h
JOIN dim_hora dh ON h.key_dim_hora = dh.key_dim_hora
GROUP BY dh.hora
ORDER BY total_servicios DESC
'''

'\nSELECT \n    dh.hora AS hora_del_dia,\n    SUM(h.hora_total) AS total_servicios\nFROM hecho_entrega_servicio_hora h\nJOIN dim_hora dh ON h.key_dim_hora = dh.key_dim_hora\nGROUP BY dh.hora\nORDER BY total_servicios DESC\n'